In [1]:
from mip import *
import pandas as pd
import matplotlib.pyplot as plt

s_df = pd.read_csv('students.csv')
s_pair_df = pd.read_csv('student_pairs.csv')


m = Model()

S = s_df.student_id.tolist()
C = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

SC = [(s,c) for s in S for c in C]

x = m.add_var_tensor((len(S),len(C)), name='x', var_type=BINARY)

C_dict = {c:i for (c,i) in zip(C,range(len(C)))}

#Each student is allocated to exactly one class
for s in S:
    m += xsum(x[s-1,C_dict[c]] for c in C) == 1

#There are 39 or 40 students in each class
for c in C:
    m += xsum(x[s-1,C_dict[c]] for s in S) >=39
    m += xsum(x[s-1,C_dict[c]] for s in S) <=40


S_male = [row.student_id for row in s_df.itertuples() if row.gender == 1]
S_female = [row.student_id for row in s_df.itertuples() if row.gender == 0]

# # of male and female students <= 20
for c in C:
    m += xsum(x[s-1,C_dict[c]] for s in S_male) <=20
    m += xsum(x[s-1,C_dict[c]] for s in S_female) <=20
    

score = {row.student_id:row.score for row in s_df.itertuples()}

# 平均点の算出
score_mean = s_df['score'].mean()
print('Ave:{:f}'.format(score_mean))

# Ave score of each class should be score_mean +- 10
for c in C:
    m += (score_mean - 10) * xsum([x[s-1,C_dict[c]] for s in S]) <= xsum([x[s-1,C_dict[c]] * score[s] for s in S])
    m += xsum([x[s-1,C_dict[c]] * score[s] for s in S]) <= (score_mean + 10) * xsum([x[s-1,C_dict[c]] for s in S])
    
# リーダー気質の生徒の集合
S_leader = [row.student_id for row in s_df.itertuples() if row.leader_flag == 1]

# (5)各クラスにリーダー気質の生徒を2人以上割り当てる。
for c in C:
    m += xsum([x[s-1,C_dict[c]] for s in S_leader]) >= 2

# 特別な支援が必要な生徒の集合
S_support = [row.student_id for row in s_df.itertuples() if row.support_flag == 1]

# (6) 特別な支援が必要な生徒は各クラスに1人以下とする。
for c in C:
    m += xsum([x[s-1,C_dict[c]] for s in S_support]) <= 1

    # 生徒の特定ペアリスト
SS = [(row.student_id1, row.student_id2) for row in s_pair_df.itertuples()]

# (7) 特定ペアの生徒は同一クラスに割り当てない。
for row in s_pair_df.itertuples():
    s1 = row.student_id1
    s2 = row.student_id2
    for c in C:
        m += x[s1-1,C_dict[c]] + x[s2-1,C_dict[c]] <= 1

Ave:303.644654


In [7]:
m.objective = minimize(xsum(xsum((x[s-1,C_dict[c]] * score[s] - score_mean) for s in S) for c in C))
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [ ]:
result_df = s_df.copy()

S2C = {s:c for s in S for c in C if x[s-1,C_dict[c]].x == 1}

result_df['assigned_class'] = result_df['student_id'].map(S2C)


fig = plt.figure(figsize=(12,20))
for i, c in enumerate(C):
    cls_df = result_df[result_df['assigned_class']==c]
    ax = fig.add_subplot(4
                         , 2
                         , i+1
                         , xlabel='score'
                         , ylabel='num'
                         , xlim=(0, 500)
                         , ylim=(0, 30)
                         , title='Class:{:s}'.format(c)
                        )
    ax.hist(cls_df['score'], bins=range(0,500,40))

In [ ]:
result_df[result_df['assigned_class'] == 'A']